In [ ]:
#### import global modules
import os
import sys
import pandas as pd
import numpy as np
from pathlib import Path
from yaml import safe_load
import google.oauth2.credentials
from google.cloud import bigquery
import gc

# Set global vars
pth_project = Path(os.getcwd().split('notebooks')[0])
pth_data = pth_project / 'data'
pth_queries = pth_project / 'core' / 'queries'
pth_creds = pth_project / 'conf' / 'local' / 'project_config.yaml'
sys.path.insert(0, str(pth_project))
d_project_config = safe_load(pth_creds.open())
# d_params = safe_load((pth_project / 'core' / 'parameters' / 'common.yaml').open())['data_extract']

# import local modules
from core.utils.gcp import connect_bq_services
# from core.etl.extract import extract_bq_data, extract_pr_codes, format_conv_df, filter_convs

# Connect to google services
bq_client = connect_bq_services(d_project_config['gcp-project-name'])
pd.options.display.max_rows = 100

In [ ]:
def extract_bq_data(bq_client, sql=None, pth_query=None):
    if sql is not None:
        df = bq_client.query(sql).to_dataframe()
    elif pth_query is not None:
        sql = pth_query.read_text()
        df = bq_client.query(sql).to_dataframe()
    else:
        raise ValueError('`sql` or `pth_query` should be set')  
    return df

In [ ]:
Query='''


select *
from `cio-datahub-enterprise-pr-183a.ent_cust_intractn.bq_ffh_dbm`
where cmpgn_cd='HBUR' 

'''

In [ ]:
DF=extract_bq_data(bq_client, sql=Query)

In [ ]:
DF.info()

In [ ]:
DF['in_hm_dt'].value_counts()

In [ ]:
DF['creative_cd'].value_counts()

In [ ]:
DF['cmpgn_typ_desc'].value_counts()

In [ ]:
pd.DataFrame(DF.groupby(['creative_cd','ctrl_flg']).agg(
    
    Customer_count= ('bus_bacct_num','nunique'),
    # # Customer_Share= ('customer_id',lambda x:x.count()*100/Merge_DF_4.shape[0])
    # Churn_total=('Telus_Churn_Flag',lambda x: x.sum()),
    # Churn_rate=('Telus_Churn_Flag',lambda x: x.mean()*100)
).reset_index())

In [ ]:
config= bigquery.job.LoadJobConfig()

# config._properties['timePartitioning'] = {'field': 'Month_Year'}
config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE

Table_BQ = 'SHS.SHS_GTM_ADC_UseCases_30Nov2022'

bq_table_instance= bq_client.load_table_from_dataframe(DF, Table_BQ,job_config=config)